# Reconnaissance Activity
Detects activity as "net user administrator /domain" and "net group domain admins /domain"

## Rule Content
```
- title: Reconnaissance Activity
  id: 968eef52-9cff-4454-8992-1e74b9cbad6c
  status: experimental
  description: Detects activity as "net user administrator /domain" and "net group
    domain admins /domain"
  references:
  - https://findingbad.blogspot.de/2017/01/hunting-what-does-it-look-like.html
  author: Florian Roth (rule), Jack Croock (method)
  tags:
  - attack.discovery
  - attack.t1087
  - attack.t1069
  - attack.s0039
  logsource:
    product: windows
    service: security
    definition: The volume of Event ID 4661 is high on Domain Controllers and therefore
      "Audit SAM" and "Audit Kernel Object" advanced audit policy settings are not
      configured in the recommendations for server systems
    category: null
  detection:
    selection:
    - EventID: 4661
      ObjectType: SAM_USER
      ObjectName: S-1-5-21-*-500
      AccessMask: '0x2d'
    - EventID: 4661
      ObjectType: SAM_GROUP
      ObjectName: S-1-5-21-*-512
      AccessMask: '0x2d'
    condition: selection
  falsepositives:
  - Administrator activity
  - Penetration tests
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4661" AND object_access_mask_requested:"0x2d" AND ((object_type:"SAM_USER" AND object_name.keyword:S\-1\-5\-21\-*\-500) OR (object_type:"SAM_GROUP" AND object_name.keyword:S\-1\-5\-21\-*\-512)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()